In [2]:

import os
import ast
import json
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
from client import HFClient
MAX_LENGTH = 4096
TRUNCATE_LENGTH = 1024

MODEL_PATH = TOKENIZER_PATH = "/data/share_user/zzd/ckpt/chatglm3-6b"
client = HFClient(MODEL_PATH, TOKENIZER_PATH, None)

/root/miniconda3/envs/zzd-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [01:48<00:00, 15.44s/it]


In [3]:
dataset =  json.load(open(f'/data/share_user/zzd/data/rlhf_data/sft_data_v4/tmp_data/weather_data_tmp.json', 'rb'))
from tqdm import tqdm
from conversation import postprocess_text, preprocess_text, Conversation, Role
from tool_registry import dispatch_tool, get_tools
tools = get_tools()

for i in tqdm(range(len(dataset))):
    history: list[Conversation] = []

    messages = dataset[i]['conversations']

    for conv in messages:
        if conv['role'] == 'user':
            history.append(Conversation(Role.USER, conv['content']))
        elif conv['role'] == 'tool':
            tool, *output_text = conv['content'].replace('location', 'city_name').strip().split('\n')
            output_text = '\n'.join(output_text)
            history.append(Conversation(Role.TOOL,postprocess_text(output_text),tool,))
        elif conv['role'] == 'observation':
            history.append(Conversation(Role.OBSERVATION, conv['content']))

        output_text = ''
        for response in client.generate_stream(
                        system=None,
                        tools=tools,
                        history=history,
                        do_sample=True,
                        max_length=MAX_LENGTH,
                        temperature=1.0,
                        top_p=0.95,
                        stop_sequences=[str(r) for r in (Role.USER, Role.OBSERVATION)],
                    ):
            output_text += response.token.text
    dataset[i]['conversations'].append({'role':'assistant', 'content':output_text})
    break


[registered tool] {'description': 'Generates a random number x, s.t. range[0] <= x < range[1]',
 'name': 'random_number_generator',
 'params': [{'description': 'The random seed used by the generator',
             'name': 'seed',
             'required': True,
             'type': 'int'},
            {'description': 'The range of the generated numbers',
             'name': 'range',
             'required': True,
             'type': 'tuple[int, int]'}]}
[registered tool] {'description': 'Get the current weather for `city_name`',
 'name': 'get_weather',
 'params': [{'description': 'The name of the city to be queried',
             'name': 'city_name',
             'required': True,
             'type': 'str'}]}


  0%|          | 0/300 [00:00<?, ?it/s]

input_shape> torch.Size([1, 292])
input_shape> torch.Size([1, 316])
input_shape> torch.Size([1, 400])


  0%|          | 0/300 [00:03<?, ?it/s]


In [6]:
dataset[0]

{'conversations': [{'role': 'user', 'content': '能否告诉我山海关区天下第一关现在的天气情况？'},
  {'role': 'tool',
   'content': "get_weather\n ```python\ntool_call(location='山海关区天下第一关')\n```"},
  {'role': 'observation',
   'content': "{'lives': {'province': '河北', 'city': '山海关区', 'temperature': '8', 'humidity': '55', 'weather': '晴', 'winddirection': '东', 'windpower': '≤3', 'reporttime': '2023-12-07 15:06:53'}}"},
  {'role': 'assistant',
   'content': '\n 山海关区天下第一关目前的天气情况为晴，气温为8摄氏度，相对湿度为55%，风向为东风，风力为≤3。这些信息是来自于天下第一关的实时天气预报，更新时间为2023年12月7日15:06:53。'}],
 'id': 0}

In [5]:
print(history)
output_text

[Conversation(role=<Role.USER: 2>, content='能否告诉我山海关区天下第一关现在的天气情况？', tool=None, image=None), Conversation(role=<Role.TOOL: 4>, content="```python\ntool_call(city_name='山海关区天下第一关')\n```", tool='get_weather', image=None), Conversation(role=<Role.OBSERVATION: 6>, content="{'lives': {'province': '河北', 'city': '山海关区', 'temperature': '8', 'humidity': '55', 'weather': '晴', 'winddirection': '东', 'windpower': '≤3', 'reporttime': '2023-12-07 15:06:53'}}", tool=None, image=None)]


'\n 山海关区天下第一关目前的天气情况为晴，气温为8摄氏度，相对湿度为55%，风向为东风，风力为≤3。这些信息是来自于天下第一关的实时天气预报，更新时间为2023年12月7日15:06:53。'

In [9]:
new_file_path = '/data/share_user/zzd/data/rlhf_data/sft_data_v4/weather_data_done.json'
with open(new_file_path, "w", encoding="utf-8") as dump_f:
    json.dump(dataset, dump_f, ensure_ascii=False)

In [1]:
from tool_registry import dispatch_tool, get_tools
tools = get_tools()
tools['get_weather']

[registered tool] {'description': 'Generates a random number x, s.t. range[0] <= x < range[1]',
 'name': 'random_number_generator',
 'params': [{'description': 'The random seed used by the generator',
             'name': 'seed',
             'required': True,
             'type': 'int'},
            {'description': 'The range of the generated numbers',
             'name': 'range',
             'required': True,
             'type': 'tuple[int, int]'}]}
[registered tool] {'description': 'Get the current weather for `city_name`',
 'name': 'get_weather',
 'params': [{'description': 'The name of the city to be queried',
             'name': 'city_name',
             'required': True,
             'type': 'str'}]}


{'name': 'get_weather',
 'description': 'Get the current weather for `city_name`',
 'params': [{'name': 'city_name',
   'description': 'The name of the city to be queried',
   'type': 'str',
   'required': True}]}

In [5]:
list(tools.keys())

['random_number_generator', 'get_weather']

In [3]:
from conversation import postprocess_text, preprocess_text, Conversation, Role


prompt_text = "查查北京的天气情况"
prompt_text = prompt_text.strip()
calling_tool = False
role = calling_tool and Role.OBSERVATION or Role.USER

history: list[Conversation] = []
history.append(Conversation(role, prompt_text))
print(history)
input_text = preprocess_text(None, tools, history,)
print(input_text)


[Conversation(role=<Role.USER: 2>, content='查查北京的天气情况', tool=None, image=None)]
<|user|> 查查北京的天气情况 None
<|system|>
Answer the following questions as best as you can. You have access to the following tools:
{"random_number_generator": {"name": "random_number_generator", "description": "Generates a random number x, s.t. range[0] <= x < range[1]", "params": [{"name": "seed", "description": "The random seed used by the generator", "type": "int", "required": true}, {"name": "range", "description": "The range of the generated numbers", "type": "tuple[int, int]", "required": true}]}, "get_weather": {"name": "get_weather", "description": "Get the current weather for `city_name`", "params": [{"name": "city_name", "description": "The name of the city to be queried", "type": "str", "required": true}]}}<|user|>
查查北京的天气情况<|assistant|>



In [4]:
output_text = ''
for response in client.generate_stream(
                system=None,
                tools=tools,
                history=history,
                do_sample=True,
                max_length=MAX_LENGTH,
                temperature=1.0,
                top_p=0.95,
                stop_sequences=[str(r) for r in (Role.USER, Role.OBSERVATION)],
            ):
    output_text += response.token.text
    print(response)
    

input_shape> torch.Size([1, 285])
TextGenerationStreamResponse(token=Token(id=0, text='get', logprob=0.0, special=False), generated_text='get', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='_', logprob=0.0, special=False), generated_text='get_', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='weather', logprob=0.0, special=False), generated_text='get_weather', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='\n', logprob=0.0, special=False), generated_text='get_weather\n', details=None)
TextGenerationStreamResponse(token=Token(id=0, text=' ```', logprob=0.0, special=False), generated_text='get_weather\n ```', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='python', logprob=0.0, special=False), generated_text='get_weather\n ```python', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='\n', logprob=0.0, special=False), generated_text='get_weather\n ```python\n', details=None)
TextGenerationStrea

In [5]:
token = response.token
print(token)
print(output_text)
match token.text.strip():
    case '<|user|>':
        history.append(Conversation(Role.ASSISTANT, postprocess_text(output_text),))
        # return
    # Initiate tool call
    case '<|assistant|>':
        history.append(Conversation(Role.ASSISTANT, postprocess_text(output_text),))
        output_text = ''
        # continue
    case '<|observation|>':
        tool, *output_text = output_text.strip().split('\n')
        output_text = '\n'.join(output_text)
        
        history.append(Conversation(Role.TOOL,postprocess_text(output_text),tool,))
print(history)

Token(id=0, text='<|observation|>', logprob=0.0, special=True)
get_weather
 ```python
tool_call(city_name='北京')
```<|observation|>
[Conversation(role=<Role.USER: 2>, content='查查北京的天气情况', tool=None, image=None), Conversation(role=<Role.TOOL: 4>, content="```python\ntool_call(city_name='北京')\n```", tool='get_weather', image=None)]


In [6]:
print(output_text)

import re
os.environ['http_proxy'] = 'http://proxygate2.ctripcorp.com:8080'
os.environ['https_proxy'] = 'http://proxygate2.ctripcorp.com:8080'

def extract_code(text: str) -> str:
    pattern = r'```([^\n]*)\n(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches[-1][1]

def tool_call(*args, **kwargs) -> dict:
    print("=== Tool call:")
    print(args)
    print(kwargs)
    return kwargs

code = extract_code(output_text)
print(code)

args = eval(code, {'tool_call': tool_call}, {})

from tool_registry import dispatch_tool
observation = dispatch_tool(tool, args)
print(observation)

history.append(Conversation(Role.OBSERVATION, observation))

 ```python
tool_call(city_name='北京')
```<|observation|>
tool_call(city_name='北京')

=== Tool call:
()
{'city_name': '北京'}
{'lives': {'province': '北京', 'city': '北京市', 'temperature': '2', 'humidity': '24', 'weather': '晴', 'winddirection': '西', 'windpower': '≤3', 'reporttime': '2023-11-30 17:40:10'}}


In [7]:
tool_call

<function __main__.tool_call(*args, **kwargs) -> dict>

In [8]:
history

[Conversation(role=<Role.USER: 2>, content='查查北京的天气情况', tool=None, image=None),
 Conversation(role=<Role.TOOL: 4>, content="```python\ntool_call(city_name='北京')\n```", tool='get_weather', image=None),
 Conversation(role=<Role.OBSERVATION: 6>, content="{'lives': {'province': '北京', 'city': '北京市', 'temperature': '2', 'humidity': '24', 'weather': '晴', 'winddirection': '西', 'windpower': '≤3', 'reporttime': '2023-11-30 17:40:10'}}", tool=None, image=None)]

In [9]:
output_text = ''
for response in client.generate_stream(
                system=None,
                tools=tools,
                history=history,
                do_sample=True,
                max_length=MAX_LENGTH,
                temperature=1.0,
                top_p=0.95,
                stop_sequences=[str(r) for r in (Role.USER, Role.OBSERVATION)],
            ):
    output_text += response.token.text
    print(response)

history.append(Conversation(Role.ASSISTANT, postprocess_text(output_text),))
print(output_text)

input_shape> torch.Size([1, 386])
TextGenerationStreamResponse(token=Token(id=0, text='\n', logprob=0.0, special=False), generated_text='\n', details=None)
TextGenerationStreamResponse(token=Token(id=0, text=' 北京', logprob=0.0, special=False), generated_text='\n 北京', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='目前的', logprob=0.0, special=False), generated_text='\n 北京目前的', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='天气', logprob=0.0, special=False), generated_text='\n 北京目前的天气', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='情况', logprob=0.0, special=False), generated_text='\n 北京目前的天气情况', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='是', logprob=0.0, special=False), generated_text='\n 北京目前的天气情况是', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='晴', logprob=0.0, special=False), generated_text='\n 北京目前的天气情况是晴', details=None)
TextGenerationStreamResponse(token=Token(id=0, text='朗', logpr

In [10]:
history

[Conversation(role=<Role.USER: 2>, content='查查北京的天气情况', tool=None, image=None),
 Conversation(role=<Role.TOOL: 4>, content="```python\ntool_call(city_name='北京')\n```", tool='get_weather', image=None),
 Conversation(role=<Role.OBSERVATION: 6>, content="{'lives': {'province': '北京', 'city': '北京市', 'temperature': '2', 'humidity': '24', 'weather': '晴', 'winddirection': '西', 'windpower': '≤3', 'reporttime': '2023-11-30 17:40:10'}}", tool=None, image=None),
 Conversation(role=<Role.ASSISTANT: 3>, content='北京目前的天气情况是晴朗的，温度为2℃，湿度为24%，风向为西风，风力为≤3。这是最新的天气情况，报告时间为2023-11-30 17:40:10。', tool=None, image=None)]